In [1]:
import pandas as pd
import requests
import json
import folium

In [2]:
with open('./observatory/jo.json', encoding='utf-8') as f:
    ObsCode = json.load(f)
with open('./observatory/bui.json', encoding='utf-8') as f:
    BuiCode = json.load(f)
ServiceKey = 'c2jVwblwmcHB5tOEWxEjbg=='

In [6]:
jo_obs = {}
for key, value in ObsCode.items():
    url = 'http://www.khoa.go.kr/api/oceangrid/tideObsRecent/search.do'
    params = {
        'ServiceKey' : ServiceKey,
        'ObsCode' : value,
        'ResultType' : 'json'
    }
    response = requests.get(url, params=params)
    result = response.json()
    print(result)
  

{'result': {'error': 'The number of requested of today is 20000 exceeded.'}}
{'result': {'error': 'The number of requested of today is 20000 exceeded.'}}
{'result': {'error': 'The number of requested of today is 20000 exceeded.'}}
{'result': {'error': 'The number of requested of today is 20000 exceeded.'}}
{'result': {'error': 'The number of requested of today is 20000 exceeded.'}}
{'result': {'error': 'The number of requested of today is 20000 exceeded.'}}
{'result': {'error': 'The number of requested of today is 20000 exceeded.'}}
{'result': {'error': 'The number of requested of today is 20000 exceeded.'}}
{'result': {'error': 'The number of requested of today is 20000 exceeded.'}}
{'result': {'error': 'The number of requested of today is 20000 exceeded.'}}
{'result': {'error': 'The number of requested of today is 20000 exceeded.'}}
{'result': {'error': 'The number of requested of today is 20000 exceeded.'}}
{'result': {'error': 'The number of requested of today is 20000 exceeded.'}}

In [ ]:
jo_obs = {}
for key, value in ObsCode.items():
    url = 'http://www.khoa.go.kr/api/oceangrid/tideObsRecent/search.do'
    params = {
        'ServiceKey' : ServiceKey,
        'ObsCode' : value,
        'ResultType' : 'json'
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        result = response.json()
        items = result['result']['data']
        meta = result['result']['meta']
        # 데이터와 메타 합치기
        all_data = {**meta, **items}
        jo_obs[key] = all_data
    else:
        print(f'조위관측소 에러: {response.status_code}, {key}')

jo_df = pd.DataFrame(jo_obs).T

KeyError: 'meta'

In [91]:
jo_df.drop('obs_last_req_cnt', axis=1, inplace=True)
jo_df.head()

,obs_post_id,obs_post_name,obs_lat,obs_lon,record_time,tide_level,water_temp,Salinity,air_temp,air_press,wind_dir,wind_speed,wind_gust,wave_hight,current_dir,current_speed
가덕도,DT_0063,가덕도,35.02417,128.81093,2025-06-30 12:57:00,150,20.8,32.7,27.0,1011.5,176,2.8,4.5,NaN,NaN,NaN
강화대교,DT_0032,강화대교,37.73194,126.52222,2025-06-30 12:57:00,363,null,null,null,1010.0,null,null,null,NaN,NaN,NaN
거문도,DT_0031,거문도,34.02833,127.30888,2025-06-30 12:57:00,261,19.7,33.0,26.8,1012.1,185,1.6,2.3,NaN,NaN,NaN
거제도,DT_0029,거제도,34.80138,128.69916,2025-06-30 12:57:00,153,17.8,33.9,23.2,1011.6,154,3.4,5.1,NaN,NaN,NaN
고흥발포,DT_0026,고흥발포,34.48111,127.34277,2025-06-30 12:57:00,272,21.7,32.8,25.0,1012.0,202,0.4,0.6,NaN,NaN,NaN


In [93]:
bui_obs = {}
for key, value in BuiCode.items():
    url = 'http://www.khoa.go.kr/api/oceangrid/buObsRecent/search.do'
    params = {
        'ServiceKey' : ServiceKey,
        'ObsCode' : value,
        'ResultType' : 'json'
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        result = response.json()
        items = result['result']['data']
        meta = result['result']['meta']
        # 데이터와 메타 합치기
        all_data = {**meta, **items}
        bui_obs[key] = all_data
    else:
        print(f'부이 에러: {response.status_code}, {key}')

bui_df = pd.DataFrame(bui_obs).T
bui_df.drop('obs_last_req_cnt', axis=1, inplace=True)

In [94]:
bui_df.head()

,obs_post_id,obs_lat,obs_post_name,obs_lon,Salinity,record_time,air_temp,water_temp,current_dir,wind_dir,wind_speed,wave_height,current_speed,air_pres
감천항,TW_0088,35.0528,감천항,129.00308,31.3,2025-06-30 13:00:00,,14.5,251,,,0.2,3.3,
경인항,TW_0077,37.52338,경인항,126.59208,,2025-06-30 13:00:00,,,,,,,,
경포대해수욕장,TW_0089,37.80897,경포대해수욕장,128.93188,34.0,2025-06-30 13:05:00,25.7,20.5,315,95,5.0,0.1,20.0,1009.5
고래불해수욕장,TW_0095,36.58,고래불해수욕장,129.45408,31.4,2025-06-30 13:05:00,25.5,20.9,335,173,6.2,0.2,10.5,1010.0
광양항,TW_0074,34.85972,광양항,127.79277,18.2,2025-06-30 13:00:00,,20.9,50,,,,1.6,


In [107]:
m = folium.Map(location=[36.5, 127.8], zoom_start=7)

for idx in jo_df.index:
    row = jo_df.loc[idx]
    try:
        lat = float(row['obs_lat'])
        lon = float(row['obs_lon'])
        popup = f"""<b>조위관측소</b><br>
          {idx}<br>
          수온: {row.get('water_temp', '')} ℃<br>
          염분: {row.get('Salinity', '')} psu<br>
          기온: {row.get('air_temp', '')} ℃<br>
          기압: {row.get('air_press', '')} hPa<br>
          풍향: {row.get('wind_dir', '')} deg<br>
          풍속: {row.get('wind_speed', '')} m/s<br>
          돌풍: {row.get('wind_gust', '')} m/s<br>
          조위: {row.get('tide_level', '')} cm<br>
          유향: {row.get('current_dir', '')} deg<br>
          유속: {row.get('current_speed', '')} m/s
          """
        folium.Marker([lat, lon], tooltip=popup,
            icon=folium.Icon(icon='home', prefix='fa', color='orange')).add_to(m)
    except Exception as e:
        print(f'조위관측소 {idx} 위치 정보 없음: {e}')

In [ ]:
for idx in bui_df.index:
    row = bui_df.loc[idx]
    try:
        lat = float(row['obs_lat'])
        lon = float(row['obs_lon'])
        popup = f"""<b>해양관측부이</b><br>
{idx}<br>
수온: {row.get('water_temp', '')} ℃<br>
염분: {row.get('Salinity', '')} psu<br>
기온: {row.get('air_temp', '')} ℃<br>
기압: {row.get('air_press', '')} hPa<br>
풍향: {row.get('wind_dir', '')} deg<br>
풍속: {row.get('wind_speed', '')} m/s<br>
유향: {row.get('current_dir', '')} deg<br>
유속: {row.get('current_speed', '')} cm/s<br>
파고: {row.get('wave_height', '')} m
"""
        folium.Marker(
            location=[lat, lon], tooltip=popup,
            icon=folium.Icon(icon='star', prefix='fa', color='blue')
        ).add_to(m)
    except Exception as e:
        print(f'부이 {idx} 위치 정보 없음: {e}')

In [101]:
m.save('./saveFiles/obs_map.html')